## Import Library

In [198]:
import pandas as pd
import openpyxl
import xlrd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from matplotlib import rcParams
import re
import calendar
import os
from natsort import natsorted

import warnings
warnings.filterwarnings('ignore')

### Setting Working directory

In [126]:
os.chdir("D:\\Work\\2021_02(Feb)\\CTB\\Data_jan21\\")

In [127]:
Startdate = "2020-01-01"
Enddate = "2021-02-01"
#Enddate = "2020-12-31"

## Functions

In [128]:
#getting granular level insights from date
def add_datepart(df, fldname, drop=True):
    
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
    attr = ['Year', 'Month', 'Week', 'Day','hour', 'Dayofweek', 'Dayofyear','weekofyear']
    
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    
    df[targ_pre + "MonthName"] = df[targ_pre + "Month"].astype(int).apply(lambda x: calendar.month_abbr[x])
    df[targ_pre + "MonthKey"] =  df[targ_pre + "Year"].astype(str)+"_"+df[targ_pre + "Month"].astype(str)
    df[targ_pre + "WeekKey"] =  df[targ_pre + "Year"].astype(str)+"_"+df[targ_pre + "Week"].astype(str)
        
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

In [129]:
def select_date_range(dataset, datevariable, startdate, enddate):
    print("old shape:", dataset.shape)
    dataset = dataset[(dataset[datevariable]>Startdate) & (dataset[datevariable]<Enddate)]
    print("new shape:", dataset.shape)
    return dataset

In [130]:
def Trend_Analysis_gb(dataset, Variables, DateVariable, GroupbyVariable, TrendType):

    Trend = dataset[Variables].groupby(GroupbyVariable,as_index=False).count()
       
    if TrendType =="Monthly":
        #Trend['Month_name']= Trend[DateVariable].astype(int).apply(lambda x: calendar.month_abbr[x])
        pass
        
    if TrendType =="Weekly":    
        pass
    
    if TrendType =="Hourly":
        Hourly_Labels = ['22-24','0-2','0-2','2-4','2-4','4-6','4-6','6-8','6-8','8-10','8-10','10-12','10-12','12-14','12-14','14-16','14-16','16-18','16-18','18-20','18-20','20-22','20-22','22-24']
        interval_range = pd.interval_range(start=0, freq=2, end=24)
        Trend['Range'] = pd.cut(Trend[DateVariable], bins=interval_range)
        Trend['Range_name']= pd.DataFrame(Hourly_Labels)
        Trend_Hr_range = Trend[['ORDER_ID','Range_name']].groupby('Range_name',as_index=False).sum()
        Trend = Trend_Hr_range.copy()
        
        
    return Trend   

In [131]:
def Trend_Analysis_pi(dataset, Variables, CountVariable,DateVariable, GroupbyVariable, TrendType):
    
    Trend = dataset[Variables].pivot_table(index=GroupbyVariable, columns=CountVariable, aggfunc=len, fill_value=0)
    Trend.reset_index(inplace=True)
        
    if TrendType =="Monthly":
        #Trend['Month_name']= Trend[DateVariable].astype(int).apply(lambda x: calendar.month_abbr[x])
        pass
        
    if TrendType =="Weekly":    
        pass
    
    if TrendType =="Hourly":
        Hourly_Labels = ['22-24','0-2','0-2','2-4','2-4','4-6','4-6','6-8','6-8','8-10','8-10','10-12','10-12','12-14','12-14','14-16','14-16','16-18','16-18','18-20','18-20','20-22','20-22','22-24']
        interval_range = pd.interval_range(start=0, freq=2, end=24)
        Trend['Range'] = pd.cut(Trend[DateVariable], bins=interval_range)
        Trend['Range_name']= pd.DataFrame(Hourly_Labels)
        Trend_Hr_range = Trend[['ORDER_ID','Range_name']].groupby('Range_name',as_index=False).sum()
        Trend = Trend_Hr_range.copy()
        
        
    return Trend
    

In [132]:
def Merge(left, left_on_var, right1, right1_on_var, right2, right2_on_var, how, ind, occurrence):
    merged1 = pd.merge(left, right1, left_on=left_on_var, right_on = right1_on_var, how = how,indicator=ind)
    if occurrence == 'double':
        merged1.drop(['_merge'], axis=1,inplace = True)
        merged2 = pd.merge(merged1, right2, left_on=left_on_var ,right_on = right2_on_var,how = how,indicator=ind)
        return merged2
    return merged1

## Read CTB Data

In [133]:
CTB_data_raw = pd.read_excel("ctb_ord_data0202.xls")
print(CTB_data_raw.shape)
CTB_data_raw.head(2)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
(64445, 33)


,_REGION _CODE,DELR_RGON_NAME,_DEALER _ID,_DEALER _NAME,_ASSIGNED _SALES _CONSULTANT,_STATE,_CITY,_ORDER _ID,_ORDER _DATE,_ORDER _STATUS,...,_BOOKING (_Y/_N),_FINAL _PURCHASE _AMOUNT,_BOOKING _AMOUNT,_BALANCE _AMOUNT,_TRADE-IN (_Y/_N),_DEALER _CODE (_Y/_N),_DEAL _CODE _VALUE,BOOKING_DATE,INVC_CNFR_YN,PAY_AT_DELR_YN
0,C1101,Central RO1,N1A03,KONCEPT HYUNDAI,Bhuvnesh Karnatak,Delhi,Delhi,ORD202012816276904,28-JAN-2020 19:50:08,Cancellation,...,N,0,0,0,N,N,0,NaT,N,N
1,C1101,Central RO1,N1A03,KONCEPT HYUNDAI,Bhuvnesh Karnatak,Delhi,Delhi,ORD20202642245061,06-FEB-2020 15:19:11,Cancellation,...,N,0,0,0,N,N,0,NaT,N,N


In [134]:
CTB_New_columns = ['REGION_CODE', 'DELR_RGON_NAME', 'DEALER_ID', 'DEALER_NAME',
       'ASSIGNED_SALES_CONSULTANT', 'STATE', 'CITY', 'ORDER_ID', 'ORDER_DATE',
       'ORDER_STATUS', 'ORDER_STAGE', 'CONSULTANT_ASSISGN_Y_N',
       'CUSTOMER_NAME', 'CUSTOMER_MOBILE', 'CUSTOMER_EMAIL', 'MODEL', 'ENGINE',
       'VARIANT', 'INTERIOR_COLOR', 'EXTERIOR_COLOR', 'FINANCE', 'LOAN_TYPE',
       'FINANCE_STATUS', 'BOOKING', 'FINAL_PURCHASE_AMOUNT', 'BOOKING_AMOUNT',
       'BALANCE_AMOUNT', 'TRADE', 'DEALER_CODE', 'DEAL_CODE_VALUE',
       'BOOKING_DATE', 'INVC_CNFR_YN', 'PAY_AT_DELR_YN']
#33columns

In [135]:
CTB_data_raw.columns=CTB_New_columns

In [136]:
CTB_data_raw.shape

(64445, 33)

In [137]:
add_datepart(CTB_data_raw,'ORDER_DATE',False)

In [138]:
print(CTB_data_raw.shape)
print(CTB_data_raw.columns)
CTB_data_raw.head()

(64445, 44)
Index(['REGION_CODE', 'DELR_RGON_NAME', 'DEALER_ID', 'DEALER_NAME',
       'ASSIGNED_SALES_CONSULTANT', 'STATE', 'CITY', 'ORDER_ID', 'ORDER_DATE',
       'ORDER_STATUS', 'ORDER_STAGE', 'CONSULTANT_ASSISGN_Y_N',
       'CUSTOMER_NAME', 'CUSTOMER_MOBILE', 'CUSTOMER_EMAIL', 'MODEL', 'ENGINE',
       'VARIANT', 'INTERIOR_COLOR', 'EXTERIOR_COLOR', 'FINANCE', 'LOAN_TYPE',
       'FINANCE_STATUS', 'BOOKING', 'FINAL_PURCHASE_AMOUNT', 'BOOKING_AMOUNT',
       'BALANCE_AMOUNT', 'TRADE', 'DEALER_CODE', 'DEAL_CODE_VALUE',
       'BOOKING_DATE', 'INVC_CNFR_YN', 'PAY_AT_DELR_YN', 'ORDER_DATEYear',
       'ORDER_DATEMonth', 'ORDER_DATEWeek', 'ORDER_DATEDay', 'ORDER_DATEhour',
       'ORDER_DATEDayofweek', 'ORDER_DATEDayofyear', 'ORDER_DATEweekofyear',
       'ORDER_DATEMonthName', 'ORDER_DATEMonthKey', 'ORDER_DATEWeekKey'],
      dtype='object')


,REGION_CODE,DELR_RGON_NAME,DEALER_ID,DEALER_NAME,ASSIGNED_SALES_CONSULTANT,STATE,CITY,ORDER_ID,ORDER_DATE,ORDER_STATUS,...,ORDER_DATEMonth,ORDER_DATEWeek,ORDER_DATEDay,ORDER_DATEhour,ORDER_DATEDayofweek,ORDER_DATEDayofyear,ORDER_DATEweekofyear,ORDER_DATEMonthName,ORDER_DATEMonthKey,ORDER_DATEWeekKey
0,C1101,Central RO1,N1A03,KONCEPT HYUNDAI,Bhuvnesh Karnatak,Delhi,Delhi,ORD202012816276904,2020-01-28 19:50:08,Cancellation,...,1,5,28,19,1,28,5,Jan,2020_1,2020_5
1,C1101,Central RO1,N1A03,KONCEPT HYUNDAI,Bhuvnesh Karnatak,Delhi,Delhi,ORD20202642245061,2020-02-06 15:19:11,Cancellation,...,2,6,6,15,3,37,6,Feb,2020_2,2020_6
2,C1101,Central RO1,N1220,FRONTIER HYUNDAI,Mandeep Singh,Delhi,Delhi,ORD202011413365197,2020-01-14 11:04:27,Cancellation,...,1,3,14,11,1,14,3,Jan,2020_1,2020_3
3,C1101,Central RO1,N1223,HIMGIRI HYUNDAI,Neeraj Singhal,Delhi,Delhi,ORD202011511731225,2020-01-15 05:48:59,Cancellation,...,1,3,15,5,2,15,3,Jan,2020_1,2020_3
4,C1101,Central RO1,N1223,HIMGIRI HYUNDAI,Neeraj Singhal,Delhi,Delhi,ORD202011550357047,2020-01-15 05:54:55,Cancellation,...,1,3,15,5,2,15,3,Jan,2020_1,2020_3


In [139]:
CTB_not4 = CTB_data_raw.loc[(CTB_data_raw['ORDER_DATEMonth'] !=4)].sort_values('ORDER_DATEMonth')
CTB_not4.shape

(52612, 44)

In [140]:
CTB_4 = CTB_data_raw.loc[(CTB_data_raw['ORDER_DATEMonth'] ==4)].sort_values('ORDER_DATEMonth')
CTB_4.shape

(11833, 44)

In [141]:
#removing sc value

In [142]:
apr_remove = pd.read_excel("D:/Work/2020_08(Sept)/CTB_Analysis_28Sept/CTB_python_data_dir/AprilAnalysis.xlsx", sheet_name=' 4 SC orders frm 1')
apr_remove.shape

(7782, 17)

In [143]:
apr_merged = pd.merge(CTB_4, apr_remove['_ORDER _ID'], left_on='ORDER_ID', right_on = '_ORDER _ID', how = 'outer',indicator=True)
apr_merged.shape

(11833, 46)

In [144]:
apr_merged['_merge'].value_counts()

both          7782
left_only     4051
right_only       0
Name: _merge, dtype: int64

In [145]:
apr_custonly = apr_merged.loc[(apr_merged['_merge'] == 'left_only')]
print(apr_custonly.shape)
#apr_custonly.head(2)

(4051, 46)


In [146]:
CTB_april = apr_custonly.copy()
CTB_april.drop(['_merge'], axis=1,inplace = True)
print (CTB_april.shape)

(4051, 45)


In [147]:
frames = [CTB_not4,CTB_april]
CTB_sc_treat = pd.concat(frames)
print(CTB_sc_treat.shape)
CTB_sc_treat.columns

(56663, 45)


Index(['REGION_CODE', 'DELR_RGON_NAME', 'DEALER_ID', 'DEALER_NAME',
       'ASSIGNED_SALES_CONSULTANT', 'STATE', 'CITY', 'ORDER_ID', 'ORDER_DATE',
       'ORDER_STATUS', 'ORDER_STAGE', 'CONSULTANT_ASSISGN_Y_N',
       'CUSTOMER_NAME', 'CUSTOMER_MOBILE', 'CUSTOMER_EMAIL', 'MODEL', 'ENGINE',
       'VARIANT', 'INTERIOR_COLOR', 'EXTERIOR_COLOR', 'FINANCE', 'LOAN_TYPE',
       'FINANCE_STATUS', 'BOOKING', 'FINAL_PURCHASE_AMOUNT', 'BOOKING_AMOUNT',
       'BALANCE_AMOUNT', 'TRADE', 'DEALER_CODE', 'DEAL_CODE_VALUE',
       'BOOKING_DATE', 'INVC_CNFR_YN', 'PAY_AT_DELR_YN', 'ORDER_DATEYear',
       'ORDER_DATEMonth', 'ORDER_DATEWeek', 'ORDER_DATEDay', 'ORDER_DATEhour',
       'ORDER_DATEDayofweek', 'ORDER_DATEDayofyear', 'ORDER_DATEweekofyear',
       'ORDER_DATEMonthName', 'ORDER_DATEMonthKey', 'ORDER_DATEWeekKey',
       '_ORDER _ID'],
      dtype='object')

In [148]:
CTB_data = select_date_range(CTB_sc_treat,"ORDER_DATE",Startdate,Enddate)

old shape: (56663, 45)
new shape: (56513, 45)


In [340]:
CTB_data.to_excel("CTB_data_tillJan_sc_rm.xlsx")

In [149]:
CTB_data.drop(['_ORDER _ID'], axis=1,inplace = True)

In [150]:
CTB_data.head()

,REGION_CODE,DELR_RGON_NAME,DEALER_ID,DEALER_NAME,ASSIGNED_SALES_CONSULTANT,STATE,CITY,ORDER_ID,ORDER_DATE,ORDER_STATUS,...,ORDER_DATEMonth,ORDER_DATEWeek,ORDER_DATEDay,ORDER_DATEhour,ORDER_DATEDayofweek,ORDER_DATEDayofyear,ORDER_DATEweekofyear,ORDER_DATEMonthName,ORDER_DATEMonthKey,ORDER_DATEWeekKey
0,C1101,Central RO1,N1A03,KONCEPT HYUNDAI,Bhuvnesh Karnatak,Delhi,Delhi,ORD202012816276904,2020-01-28 19:50:08,Cancellation,...,1,5,28,19,1,28,5,Jan,2020_1,2020_5
60802,N1101,North RO1,N5211,AM HYUNDAI,Sahil Kharak,Jammu & Kashmir,Jammu,ORD2021010316655097,2021-01-03 22:23:23,Open,...,1,53,3,22,6,3,53,Jan,2021_1,2021_53
60803,C1103,Central RO3,N3239,PAWAN HYUNDAI,Manoj Kumar,Uttarpradesh,Ghaziabad,ORD2021010312206772,2021-01-03 23:43:11,Open,...,1,53,3,23,6,3,53,Jan,2021_1,2021_53
60804,N1103,North RO3,N2201,SAMTA KARNAL,Neeraj Kumar,Haryana,Karnal,ORD2021010818811684,2021-01-08 21:31:06,Open,...,1,1,8,21,4,8,1,Jan,2021_1,2021_1
60805,W1103,West RO3,W2262,KRISHOM AUTOMOTIVE LLP,Shivraj Pawar,Maharashtra,Pune,ORD2021010819740546,2021-01-08 22:47:17,Open,...,1,1,8,22,4,8,1,Jan,2021_1,2021_1


In [151]:
CTB_booking = CTB_data[CTB_data["BOOKING"]=="Y"]
print("CTB_booking : ", CTB_booking.shape)
CTB_booking_na = CTB_booking[CTB_booking["BOOKING_DATE"].notna()]
print("CTB_booking_na : ",CTB_booking_na.shape)

CTB_booking :  (1753, 44)
CTB_booking_na :  (1736, 44)


In [152]:
add_datepart(CTB_booking_na,'BOOKING_DATE',False)

In [153]:
CTB_booking_na.columns

Index(['REGION_CODE', 'DELR_RGON_NAME', 'DEALER_ID', 'DEALER_NAME',
       'ASSIGNED_SALES_CONSULTANT', 'STATE', 'CITY', 'ORDER_ID', 'ORDER_DATE',
       'ORDER_STATUS', 'ORDER_STAGE', 'CONSULTANT_ASSISGN_Y_N',
       'CUSTOMER_NAME', 'CUSTOMER_MOBILE', 'CUSTOMER_EMAIL', 'MODEL', 'ENGINE',
       'VARIANT', 'INTERIOR_COLOR', 'EXTERIOR_COLOR', 'FINANCE', 'LOAN_TYPE',
       'FINANCE_STATUS', 'BOOKING', 'FINAL_PURCHASE_AMOUNT', 'BOOKING_AMOUNT',
       'BALANCE_AMOUNT', 'TRADE', 'DEALER_CODE', 'DEAL_CODE_VALUE',
       'BOOKING_DATE', 'INVC_CNFR_YN', 'PAY_AT_DELR_YN', 'ORDER_DATEYear',
       'ORDER_DATEMonth', 'ORDER_DATEWeek', 'ORDER_DATEDay', 'ORDER_DATEhour',
       'ORDER_DATEDayofweek', 'ORDER_DATEDayofyear', 'ORDER_DATEweekofyear',
       'ORDER_DATEMonthName', 'ORDER_DATEMonthKey', 'ORDER_DATEWeekKey',
       'BOOKING_DATEYear', 'BOOKING_DATEMonth', 'BOOKING_DATEWeek',
       'BOOKING_DATEDay', 'BOOKING_DATEhour', 'BOOKING_DATEDayofweek',
       'BOOKING_DATEDayofyear', 'BOOKING_

## Read GDMS Data

In [154]:
GDMS_data_raw = pd.read_excel("ctb_gdms_data0202.xls")
GDMS_data_raw.head(2)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,CSPM_DLR_NAME,EQRY_DLR_NO,EQRY_ENQRY_NO,EQRY_DATE,EQRY_STAT,EQRY_STATUS_NM,EQCR_MODEL_CODE,EQCR_MODEL_NAME,EQRY_NEWCAR_YN,EQRY_REF_NO,CTB_BOOKING_Y,RETAIL_DATE,INVC_CNFM_YN,BKNG_NO,BKNG_DATE
0,ROSHAN HYUNDAI,N4A06,E202041462,20201104,80,Invoice Cancel,SV,All New i20,Y,ORD2020110413233196,Y,NaN,NaN,B202001236,20201104.0
1,SAI AUTO HYUNDAI,W1210,E202039463,20201028,80,Invoice Cancel,SV,All New i20,Y,ORD2020102811922815,Y,NaN,NaN,B202016328,20201028.0


In [155]:
print("Dateset Shape :" , GDMS_data_raw.shape)
print("Dateset Columns :" ,GDMS_data_raw.columns)

Dateset Shape : (62327, 15)
Dateset Columns : Index(['CSPM_DLR_NAME', 'EQRY_DLR_NO', 'EQRY_ENQRY_NO', 'EQRY_DATE',
       'EQRY_STAT', 'EQRY_STATUS_NM', 'EQCR_MODEL_CODE', 'EQCR_MODEL_NAME',
       'EQRY_NEWCAR_YN', 'EQRY_REF_NO', 'CTB_BOOKING_Y', 'RETAIL_DATE',
       'INVC_CNFM_YN', 'BKNG_NO', 'BKNG_DATE'],
      dtype='object')


In [156]:
GDMS_booking_raw = GDMS_data_raw.copy()
GDMS_booking_raw = GDMS_booking_raw[GDMS_booking_raw["BKNG_DATE"].notna()]
GDMS_booking_raw.shape

(4393, 15)

In [157]:
GDMS_booking_raw['BKNG_DATE_NEW'] = GDMS_booking_raw['BKNG_DATE'].map(lambda x: str(x)[0:4]) + "-" +GDMS_booking_raw['BKNG_DATE'].map(lambda x: str(x)[4:6])+ "-" +GDMS_booking_raw['BKNG_DATE'].map(lambda x: str(x)[6:8])
GDMS_booking_raw['BKNG_DATE_NEW'].head(2)

0    2020-11-04
1    2020-10-28
Name: BKNG_DATE_NEW, dtype: object

In [158]:
add_datepart(GDMS_booking_raw,'BKNG_DATE_NEW',False)
print(GDMS_booking.columns)
GDMS_booking = select_date_range(GDMS_booking_raw,"BKNG_DATE_NEW",Startdate,Enddate)

Index(['CSPM_DLR_NAME', 'EQRY_DLR_NO', 'EQRY_ENQRY_NO', 'EQRY_DATE',
       'EQRY_STAT', 'EQRY_STATUS_NM', 'EQCR_MODEL_CODE', 'EQCR_MODEL_NAME',
       'EQRY_NEWCAR_YN', 'EQRY_REF_NO', 'CTB_BOOKING_Y', 'RETAIL_DATE',
       'INVC_CNFM_YN', 'BKNG_NO', 'BKNG_DATE', 'BKNG_DATE_NEW',
       'BKNG_DATE_NEWYear', 'BKNG_DATE_NEWMonth', 'BKNG_DATE_NEWWeek',
       'BKNG_DATE_NEWDay', 'BKNG_DATE_NEWhour', 'BKNG_DATE_NEWDayofweek',
       'BKNG_DATE_NEWDayofyear', 'BKNG_DATE_NEWweekofyear',
       'BKNG_DATE_NEWMonthName', 'BKNG_DATE_NEWKey'],
      dtype='object')
old shape: (4393, 27)
new shape: (4381, 27)


In [342]:
#CTB and GDMS Merge Data

In [344]:
CTB_GDMS = pd.merge(CTB_data_raw, GDMS_booking, left_on=['ORDER_ID'],right_on = ['EQRY_REF_NO'], how = 'outer',indicator=True)
print(CTB_GDMS.shape)
CTB_GDMS['_merge'].value_counts()

(64454, 72)


left_only     60049
both           4396
right_only        9
Name: _merge, dtype: int64

In [345]:
#Retail data
CTB_GDMS_Both_book = CTB_GDMS[CTB_GDMS["_merge"]!="left_only"]
print(CTB_GDMS_Both_book.shape)
CTB_GDMS_Both_book.to_excel("CTB_GDMS_booking_till_jan.xlsx")

(4405, 72)


In [160]:
GDMS_retail_raw = GDMS_data_raw[(GDMS_data_raw["RETAIL_DATE"].notna()) & (GDMS_data_raw['EQRY_STAT']==45)]
print(GDMS_retail_raw.shape)
GDMS_retail_raw.head(2)

(1843, 15)


,CSPM_DLR_NAME,EQRY_DLR_NO,EQRY_ENQRY_NO,EQRY_DATE,EQRY_STAT,EQRY_STATUS_NM,EQCR_MODEL_CODE,EQCR_MODEL_NAME,EQRY_NEWCAR_YN,EQRY_REF_NO,CTB_BOOKING_Y,RETAIL_DATE,INVC_CNFM_YN,BKNG_NO,BKNG_DATE
131,HIMGIRI HYUNDAI,N2226,E202000837,20200118,45,Retail,C7,New i20,Y,ORD202011822130451,N,20200131.0,Y,B202018083,20200129.0
145,JOSHI AUTOMOBILES PVT. LTD.,N7204,E202020775,20200627,45,Retail,FH,New Creta,Y,ORD2020062713739086,Y,20200707.0,Y,B202003559,20200627.0


In [161]:
GDMS_retail_raw['RETAIL_DATE_NEW'] = GDMS_retail_raw['RETAIL_DATE'].map(lambda x: str(x)[0:4]) + "-" +GDMS_retail_raw['RETAIL_DATE'].map(lambda x: str(x)[4:6])+ "-" +GDMS_retail_raw['RETAIL_DATE'].map(lambda x: str(x)[6:8])
GDMS_retail_raw['RETAIL_DATE_NEW'].head(2)

131    2020-01-31
145    2020-07-07
Name: RETAIL_DATE_NEW, dtype: object

In [162]:
add_datepart(GDMS_retail_raw,'RETAIL_DATE_NEW',False)
print(GDMS_retail_raw.columns)
GDMS_retail = select_date_range(GDMS_retail_raw,"RETAIL_DATE_NEW",Startdate,Enddate)

Index(['CSPM_DLR_NAME', 'EQRY_DLR_NO', 'EQRY_ENQRY_NO', 'EQRY_DATE',
       'EQRY_STAT', 'EQRY_STATUS_NM', 'EQCR_MODEL_CODE', 'EQCR_MODEL_NAME',
       'EQRY_NEWCAR_YN', 'EQRY_REF_NO', 'CTB_BOOKING_Y', 'RETAIL_DATE',
       'INVC_CNFM_YN', 'BKNG_NO', 'BKNG_DATE', 'RETAIL_DATE_NEW',
       'RETAIL_DATE_NEWYear', 'RETAIL_DATE_NEWMonth', 'RETAIL_DATE_NEWWeek',
       'RETAIL_DATE_NEWDay', 'RETAIL_DATE_NEWhour', 'RETAIL_DATE_NEWDayofweek',
       'RETAIL_DATE_NEWDayofyear', 'RETAIL_DATE_NEWweekofyear',
       'RETAIL_DATE_NEWMonthName', 'RETAIL_DATE_NEWMonthKey',
       'RETAIL_DATE_NEWWeekKey'],
      dtype='object')
old shape: (1843, 27)
new shape: (1840, 27)


## Execution

### Monthly/Weekly/Hourly Analysis

In [163]:
#Monthly Enquiry trend
Monthly_trend_Enq = Trend_Analysis_gb(CTB_data,['ORDER_ID','ORDER_DATEMonthKey'],'ORDER_DATEMonthKey','ORDER_DATEMonthKey','Monthly')
Monthly_trend_Enq.columns = ["ORDER_DATEMonthKey","Enquiry Count"]

#Monthly Booking trend
Monthly_trend_Booking = Trend_Analysis_pi(GDMS_booking,['CTB_BOOKING_Y','BKNG_DATE_NEWMonthKey'],'CTB_BOOKING_Y','BKNG_DATE_NEWMonthKey','BKNG_DATE_NEWMonthKey','Monthly')
Monthly_trend_Booking.columns = ['BKNG_DATE_NEWMonthKey', 'CTB_BOOKING_N', 'CTB_BOOKING_Y']

#Monthly Retail trend
Monthly_trend_Retail = Trend_Analysis_gb(GDMS_retail,['EQRY_REF_NO','RETAIL_DATE_NEWMonthKey'],'RETAIL_DATE_NEWMonthKey','RETAIL_DATE_NEWMonthKey','Monthly')
Monthly_trend_Retail.columns = ["RETAIL_DATE_NEWMonthKey","Retail Count"]


In [164]:
Monthly_trend_Enq

,ORDER_DATEMonthKey,Enquiry Count
0,2020_1,189
1,2020_10,8590
2,2020_11,6887
3,2020_12,6602
4,2020_2,350
5,2020_3,688
6,2020_4,4051
7,2020_5,2224
8,2020_6,5888
9,2020_7,4902


In [165]:
Monthly_trend_Booking

,BKNG_DATE_NEWMonthKey,CTB_BOOKING_N,CTB_BOOKING_Y
0,2020_1,1,0
1,2020_10,416,535
2,2020_11,403,361
3,2020_12,383,182
4,2020_2,8,0
5,2020_3,31,8
6,2020_4,9,50
7,2020_5,50,40
8,2020_6,162,67
9,2020_7,244,73


In [166]:
Monthly_trend_Retail

,RETAIL_DATE_NEWMonthKey,Retail Count
0,2020_1,1
1,2020_10,273
2,2020_11,295
3,2020_12,297
4,2020_2,3
5,2020_3,2
6,2020_5,30
7,2020_6,106
8,2020_7,159
9,2020_8,219


In [175]:
#Weekly Enquiry trend
Weekly_trend_Enq = Trend_Analysis_gb(CTB_data,['ORDER_ID','ORDER_DATEWeekKey'],'ORDER_DATEWeekKey','ORDER_DATEWeekKey','Weekly')
Weekly_trend_Enq.columns = ['ORDER_DATEWeekKey','Enquiry Count']

#Weekly Booking trend
Weekly_trend_Booking = Trend_Analysis_pi(GDMS_booking,['CTB_BOOKING_Y','BKNG_DATE_NEWWeekKey'],'CTB_BOOKING_Y','BKNG_DATE_NEWWeekKey','BKNG_DATE_NEWWeekKey','Weekly')
Weekly_trend_Booking.columns = ['BKNG_DATE_NEWWeekKey', 'CTB_BOOKING_N', 'CTB_BOOKING_Y']

#Weekly Retail trend
Weekly_trend_Retail = Trend_Analysis_gb(GDMS_retail,['EQRY_REF_NO','RETAIL_DATE_NEWWeekKey'],'RETAIL_DATE_NEWWeekKey','RETAIL_DATE_NEWWeekKey','Weekly')
Weekly_trend_Retail.columns = ["RETAIL_DATE_NEWWeekKey","Retail Count"]


In [177]:
#Hourly_trend_booking
Hourly_trend_Enq = Trend_Analysis_gb(CTB_data,['ORDER_ID','ORDER_DATEhour'],'ORDER_DATEhour','ORDER_DATEhour','Hourly')
Hourly_trend_Enq.columns = ['Range_name','Enquiry Count']

Hourly_trend_booking = Trend_Analysis_gb(CTB_booking_na,['ORDER_ID','BOOKING_DATEhour'],'BOOKING_DATEhour','BOOKING_DATEhour','Hourly')
Hourly_trend_booking.columns = ["Range_name","Booking Count"]


In [176]:
Weekly_trend_Enq

,ORDER_DATEWeekKey,Enquiry Count
0,2020_10,105
1,2020_11,102
2,2020_12,160
3,2020_13,255
4,2020_14,287
5,2020_15,1175
6,2020_16,697
7,2020_17,1534
8,2020_18,634
9,2020_19,452


In [178]:
Weekly_trend_Booking

,BKNG_DATE_NEWWeekKey,CTB_BOOKING_N,CTB_BOOKING_Y
0,2020_10,9,0
1,2020_11,10,2
2,2020_12,7,3
3,2020_13,5,3
4,2020_14,0,6
5,2020_15,2,11
6,2020_16,3,7
7,2020_17,1,16
8,2020_18,3,12
9,2020_19,9,17


In [179]:
Weekly_trend_Retail

,RETAIL_DATE_NEWWeekKey,Retail Count
0,2020_12,2
1,2020_19,2
2,2020_20,5
3,2020_21,9
4,2020_22,14
5,2020_23,16
6,2020_24,21
7,2020_25,34
8,2020_26,24
9,2020_27,24


In [180]:
Hourly_trend_Enq

,Range_name,Enquiry Count
0,0-2,2249
1,10-12,6805
2,12-14,6869
3,14-16,7081
4,16-18,6150
5,18-20,6605
6,2-4,839
7,20-22,6693
8,22-24,5437
9,4-6,924


In [181]:
Hourly_trend_booking

,Range_name,Booking Count
0,0-2,23
1,10-12,256
2,12-14,259
3,14-16,248
4,16-18,249
5,18-20,205
6,2-4,10
7,20-22,181
8,22-24,96
9,4-6,11


In [232]:
Monthly_Trend_merg = Merge(Monthly_trend_Enq,'ORDER_DATEMonthKey',Monthly_trend_Booking,'BKNG_DATE_NEWMonthKey',Monthly_trend_Retail,'RETAIL_DATE_NEWMonthKey','outer',True,'double')
Monthly_Trend_merg[['Year','Month']] = Monthly_Trend_merg.ORDER_DATEMonthKey.str.split("_",expand=True)

Monthly_Trend_merg

,ORDER_DATEMonthKey,Enquiry Count,BKNG_DATE_NEWMonthKey,CTB_BOOKING_N,CTB_BOOKING_Y,RETAIL_DATE_NEWMonthKey,Retail Count,_merge,Year,Month
0,2020_1,189,2020_1,1,0,2020_1,1.0,both,2020,1
1,2020_10,8590,2020_10,416,535,2020_10,273.0,both,2020,10
2,2020_11,6887,2020_11,403,361,2020_11,295.0,both,2020,11
3,2020_12,6602,2020_12,383,182,2020_12,297.0,both,2020,12
4,2020_2,350,2020_2,8,0,2020_2,3.0,both,2020,2
5,2020_3,688,2020_3,31,8,2020_3,2.0,both,2020,3
6,2020_4,4051,2020_4,9,50,NaN,NaN,left_only,2020,4
7,2020_5,2224,2020_5,50,40,2020_5,30.0,both,2020,5
8,2020_6,5888,2020_6,162,67,2020_6,106.0,both,2020,6
9,2020_7,4902,2020_7,244,73,2020_7,159.0,both,2020,7


In [233]:
Monthly_Trend = Monthly_Trend_merg[["ORDER_DATEMonthKey","Year","Month","Enquiry Count","CTB_BOOKING_Y","CTB_BOOKING_N","Retail Count"]]
Monthly_Trend

,ORDER_DATEMonthKey,Year,Month,Enquiry Count,CTB_BOOKING_Y,CTB_BOOKING_N,Retail Count
0,2020_1,2020,1,189,0,1,1.0
1,2020_10,2020,10,8590,535,416,273.0
2,2020_11,2020,11,6887,361,403,295.0
3,2020_12,2020,12,6602,182,383,297.0
4,2020_2,2020,2,350,0,8,3.0
5,2020_3,2020,3,688,8,31,2.0
6,2020_4,2020,4,4051,50,9,NaN
7,2020_5,2020,5,2224,40,50,30.0
8,2020_6,2020,6,5888,67,162,106.0
9,2020_7,2020,7,4902,73,244,159.0


In [235]:
Monthly_Trend_final_added = Monthly_Trend.copy()
Monthly_Trend_final_added["CRM Booking"] = 0
Monthly_Trend_final_added["CRM Booking"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_4"), 114,Monthly_Trend_final_added["CRM Booking"])
Monthly_Trend_final_added["CRM Booking"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_5"), 156,Monthly_Trend_final_added["CRM Booking"])
Monthly_Trend_final_added["CRM Booking"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_6"), 122,Monthly_Trend_final_added["CRM Booking"])

Monthly_Trend_final_added["CRM Retail"] = 0
Monthly_Trend_final_added["CRM Retail"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_4"), 1,Monthly_Trend_final_added["CRM Retail"])
Monthly_Trend_final_added["CRM Retail"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_5"), 66,Monthly_Trend_final_added["CRM Retail"])
Monthly_Trend_final_added["CRM Retail"] = np.where((Monthly_Trend_final_added["ORDER_DATEMonthKey"] =="2020_6"), 112,Monthly_Trend_final_added["CRM Retail"])

Monthly_Trend_final_added["Online Booking"] = Monthly_Trend_final_added["CTB_BOOKING_Y"]+Monthly_Trend_final_added["CRM Booking"]
Monthly_Trend_final_added["Total CTB Booking"] = Monthly_Trend_final_added["CTB_BOOKING_Y"]+Monthly_Trend_final_added["CTB_BOOKING_N"]
Monthly_Trend_final_added["Total Booking"] = Monthly_Trend_final_added["CTB_BOOKING_N"]+ Monthly_Trend_final_added["CTB_BOOKING_Y"]+Monthly_Trend_final_added["CRM Booking"]
Monthly_Trend_final_added["Total Retail"] = Monthly_Trend_final_added["Retail Count"]+ Monthly_Trend_final_added["CRM Retail"]
Monthly_Trend_final_added.fillna(0.0,inplace=True)
Monthly_Trend_final_added["Retail Conversion Ratio"] = (Monthly_Trend_final_added["Total Retail"]/Monthly_Trend_final_added["Enquiry Count"])
Monthly_Trend_final_added
#np.where((CTB_Registrations['Month_Name'] == 'Apr') & (CTB_Registrations['year'] == 2020 ) , 4487,CTB_Registrations['Registration_Count'] )

,ORDER_DATEMonthKey,Year,Month,Enquiry Count,CTB_BOOKING_Y,CTB_BOOKING_N,Retail Count,CRM Booking,CRM Retail,Online Booking,Total CTB Booking,Total Booking,Total Retail,Retail Conversion Ratio
0,2020_1,2020,1,189,0,1,1.0,0,0,0,1,1,1.0,0.005291
1,2020_10,2020,10,8590,535,416,273.0,0,0,535,951,951,273.0,0.031781
2,2020_11,2020,11,6887,361,403,295.0,0,0,361,764,764,295.0,0.042834
3,2020_12,2020,12,6602,182,383,297.0,0,0,182,565,565,297.0,0.044986
4,2020_2,2020,2,350,0,8,3.0,0,0,0,8,8,3.0,0.008571
5,2020_3,2020,3,688,8,31,2.0,0,0,8,39,39,2.0,0.002907
6,2020_4,2020,4,4051,50,9,0.0,114,1,164,59,173,0.0,0.000000
7,2020_5,2020,5,2224,40,50,30.0,156,66,196,90,246,96.0,0.043165
8,2020_6,2020,6,5888,67,162,106.0,122,112,189,229,351,218.0,0.037024
9,2020_7,2020,7,4902,73,244,159.0,0,0,73,317,317,159.0,0.032436


In [237]:
monthlist = ["ORDER_DATEMonthKey","Year","Month","Enquiry Count","CTB_BOOKING_Y","CTB_BOOKING_N","Total CTB Booking","CRM Booking","Online Booking","Total Booking","CRM Retail","Total Retail","Retail Conversion Ratio"]
Monthly_Trend_final = Monthly_Trend_final_added[monthlist]
Monthly_Trend_final

,ORDER_DATEMonthKey,Year,Month,Enquiry Count,CTB_BOOKING_Y,CTB_BOOKING_N,Total CTB Booking,CRM Booking,Online Booking,Total Booking,CRM Retail,Total Retail,Retail Conversion Ratio
0,2020_1,2020,1,189,0,1,1,0,0,1,0,1.0,0.005291
1,2020_10,2020,10,8590,535,416,951,0,535,951,0,273.0,0.031781
2,2020_11,2020,11,6887,361,403,764,0,361,764,0,295.0,0.042834
3,2020_12,2020,12,6602,182,383,565,0,182,565,0,297.0,0.044986
4,2020_2,2020,2,350,0,8,8,0,0,8,0,3.0,0.008571
5,2020_3,2020,3,688,8,31,39,0,8,39,0,2.0,0.002907
6,2020_4,2020,4,4051,50,9,59,114,164,173,1,0.0,0.000000
7,2020_5,2020,5,2224,40,50,90,156,196,246,66,96.0,0.043165
8,2020_6,2020,6,5888,67,162,229,122,189,351,112,218.0,0.037024
9,2020_7,2020,7,4902,73,244,317,0,73,317,0,159.0,0.032436


In [239]:
Weekly_Trend_final = Merge(Weekly_trend_Enq,'ORDER_DATEWeekKey',Weekly_trend_Booking,'BKNG_DATE_NEWWeekKey',Weekly_trend_Retail,'RETAIL_DATE_NEWWeekKey','outer',True,'double')
Weekly_Trend_final

,ORDER_DATEWeekKey,Enquiry Count,BKNG_DATE_NEWWeekKey,CTB_BOOKING_N,CTB_BOOKING_Y,RETAIL_DATE_NEWWeekKey,Retail Count,_merge
0,2020_10,105,2020_10,9.0,0.0,NaN,NaN,left_only
1,2020_11,102,2020_11,10.0,2.0,NaN,NaN,left_only
2,2020_12,160,2020_12,7.0,3.0,2020_12,2.0,both
3,2020_13,255,2020_13,5.0,3.0,NaN,NaN,left_only
4,2020_14,287,2020_14,0.0,6.0,NaN,NaN,left_only
5,2020_15,1175,2020_15,2.0,11.0,NaN,NaN,left_only
6,2020_16,697,2020_16,3.0,7.0,NaN,NaN,left_only
7,2020_17,1534,2020_17,1.0,16.0,NaN,NaN,left_only
8,2020_18,634,2020_18,3.0,12.0,NaN,NaN,left_only
9,2020_19,452,2020_19,9.0,17.0,2020_19,2.0,both


In [ ]:
#Weekly Final variable

In [240]:
Weekly_Trend_final = Weekly_Trend_final[["ORDER_DATEWeekKey","Enquiry Count","CTB_BOOKING_Y","CTB_BOOKING_N","Retail Count"]]
Weekly_Trend_final

,ORDER_DATEWeekKey,Enquiry Count,CTB_BOOKING_Y,CTB_BOOKING_N,Retail Count
0,2020_10,105,0.0,9.0,NaN
1,2020_11,102,2.0,10.0,NaN
2,2020_12,160,3.0,7.0,2.0
3,2020_13,255,3.0,5.0,NaN
4,2020_14,287,6.0,0.0,NaN
5,2020_15,1175,11.0,2.0,NaN
6,2020_16,697,7.0,3.0,NaN
7,2020_17,1534,16.0,1.0,NaN
8,2020_18,634,12.0,3.0,NaN
9,2020_19,452,17.0,9.0,2.0


In [245]:
Weekly_Trend_final1 = Weekly_Trend_final.copy()
Weekly_Trend_final1["CRM Booking"] = 0
Weekly_Trend_final1["CRM Retail"] = 0
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_14"), 22,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_15"), 23,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_16"), 23,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_17"), 23,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_18"), 23,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_19"), 39,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_20"), 39,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_21"), 39,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_22"), 39,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_23"), 30,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_24"), 30,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_25"), 30,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_26"), 30,Weekly_Trend_final1["CRM Booking"])
Weekly_Trend_final1["CRM Booking"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_27"), 2,Weekly_Trend_final1["CRM Booking"])

Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_16"), 1,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_19"), 16,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_20"), 17,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_21"), 17,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_22"), 16,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_23"), 26,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_24"), 26,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_25"), 26,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_26"), 26,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1["CRM Retail"] = np.where((Weekly_Trend_final1["ORDER_DATEWeekKey"] =="2020_27"), 8,Weekly_Trend_final1["CRM Retail"])
Weekly_Trend_final1.fillna(0, inplace = True)
Weekly_Trend_final1["Total Booking"] = Weekly_Trend_final1["CTB_BOOKING_N"]+ Weekly_Trend_final1["CTB_BOOKING_Y"]+Weekly_Trend_final1["CRM Booking"]
Weekly_Trend_final1["Total Retail"] = Weekly_Trend_final1["Retail Count"]+ Weekly_Trend_final1["CRM Retail"]
Weekly_Trend_final1["Retail Conversion Ratio"] = (Weekly_Trend_final1["Total Retail"]/Weekly_Trend_final1["Enquiry Count"])

Weekly_Trend_final1

,ORDER_DATEWeekKey,Enquiry Count,CTB_BOOKING_Y,CTB_BOOKING_N,Retail Count,CRM Booking,CRM Retail,Total Booking,Total Retail,Retail Conversion Ratio
0,2020_10,105,0.0,9.0,0.0,0,0,9.0,0.0,0.000000
1,2020_11,102,2.0,10.0,0.0,0,0,12.0,0.0,0.000000
2,2020_12,160,3.0,7.0,2.0,0,0,10.0,2.0,0.012500
3,2020_13,255,3.0,5.0,0.0,0,0,8.0,0.0,0.000000
4,2020_14,287,6.0,0.0,0.0,22,0,28.0,0.0,0.000000
5,2020_15,1175,11.0,2.0,0.0,23,0,36.0,0.0,0.000000
6,2020_16,697,7.0,3.0,0.0,23,1,33.0,1.0,0.001435
7,2020_17,1534,16.0,1.0,0.0,23,0,40.0,0.0,0.000000
8,2020_18,634,12.0,3.0,0.0,23,0,38.0,0.0,0.000000
9,2020_19,452,17.0,9.0,2.0,39,16,65.0,18.0,0.039823


In [ ]:
#Hourly Final variable

In [246]:
Hourly_Trend_final = Merge(Hourly_trend_Enq,'Range_name',Hourly_trend_booking,'Range_name','','','outer',True,'single')
Hourly_Trend_final = Hourly_Trend_final[["Range_name","Enquiry Count","Booking Count"]]
Hourly_Trend_final

,Range_name,Enquiry Count,Booking Count
0,0-2,2249,23
1,10-12,6805,256
2,12-14,6869,259
3,14-16,7081,248
4,16-18,6150,249
5,18-20,6605,205
6,2-4,839,10
7,20-22,6693,181
8,22-24,5437,96
9,4-6,924,11


In [248]:
Hourly_Trend_final1 = Hourly_Trend_final.copy()
total_enquiry =Hourly_Trend_final1["Enquiry Count"].sum()
total_Booking =Hourly_Trend_final1["Booking Count"].sum()
print(total_enquiry)
print(total_Booking)
Hourly_Trend_final1["Enquiry Ratio"] = (Hourly_Trend_final1["Enquiry Count"]/total_enquiry)
Hourly_Trend_final1["Booking Ratio"] = (Hourly_Trend_final1["Booking Count"]/total_Booking)

Hourly_Trend_final1

56513
1736


,Range_name,Enquiry Count,Booking Count,Enquiry Ratio,Booking Ratio
0,0-2,2249,23,0.039796,0.013249
1,10-12,6805,256,0.120415,0.147465
2,12-14,6869,259,0.121547,0.149194
3,14-16,7081,248,0.125299,0.142857
4,16-18,6150,249,0.108825,0.143433
5,18-20,6605,205,0.116876,0.118088
6,2-4,839,10,0.014846,0.005760
7,20-22,6693,181,0.118433,0.104263
8,22-24,5437,96,0.096208,0.055300
9,4-6,924,11,0.016350,0.006336


### Region level Analysis

In [249]:
Region_enq = Trend_Analysis_gb(CTB_data,['ORDER_ID','DELR_RGON_NAME'],'','DELR_RGON_NAME','Weekly')
print("total enquiry : ",Region_enq.ORDER_ID.sum())
Region_enq

total enquiry :  56513


,DELR_RGON_NAME,ORDER_ID
0,Central RO1,4260
1,Central RO2,4952
2,Central RO3,3845
3,East RO1,2691
4,East RO2,1032
5,East RO3,1978
6,North RO1,2140
7,North RO2,2393
8,North RO3,2653
9,South RO1,3207


In [250]:
Region_list = CTB_data[["DELR_RGON_NAME","DEALER_ID"]]
Region_list_uniq = Region_list.drop_duplicates(subset=['DELR_RGON_NAME', 'DEALER_ID'], keep='last')
Region_list_uniq.head(2)

,DELR_RGON_NAME,DEALER_ID
45227,South RO4,S8823
63460,South RO3,S7713


In [251]:
GDMS_booking.shape

(4381, 27)

In [252]:
GDMS_booking_add_region = pd.merge(GDMS_booking, Region_list_uniq, left_on=['EQRY_DLR_NO'],right_on = ['DEALER_ID'], how = 'outer',indicator=True)
print("GDMS_add_region" , GDMS_booking_add_region.shape)
GDMS_booking_add_region['_merge'].value_counts()

GDMS_add_region (4715, 30)


both          4381
right_only     334
left_only        0
Name: _merge, dtype: int64

In [253]:
GDMS_booking_add_region.loc[GDMS_booking_add_region['_merge'] == "left_only", ['DELR_RGON_NAME']] = 'North RO3'
GDMS_booking_add_region[GDMS_booking_add_region["_merge"]=="left_only"]

,CSPM_DLR_NAME,EQRY_DLR_NO,EQRY_ENQRY_NO,EQRY_DATE,EQRY_STAT,EQRY_STATUS_NM,EQCR_MODEL_CODE,EQCR_MODEL_NAME,EQRY_NEWCAR_YN,EQRY_REF_NO,...,BKNG_DATE_NEWhour,BKNG_DATE_NEWDayofweek,BKNG_DATE_NEWDayofyear,BKNG_DATE_NEWweekofyear,BKNG_DATE_NEWMonthName,BKNG_DATE_NEWMonthKey,BKNG_DATE_NEWWeekKey,DELR_RGON_NAME,DEALER_ID,_merge


In [254]:
GDMS_booking_add_region_bl = GDMS_booking_add_region[GDMS_booking_add_region["_merge"]!="right_only"]
GDMS_booking_add_region_bl.shape

(4381, 30)

In [255]:
Region_Booking = Trend_Analysis_gb(GDMS_booking_add_region_bl,['EQRY_REF_NO','DELR_RGON_NAME'],'','DELR_RGON_NAME','Monthly')
Region_Booking

,DELR_RGON_NAME,EQRY_REF_NO
0,Central RO1,302
1,Central RO2,359
2,Central RO3,290
3,East RO1,218
4,East RO2,69
5,East RO3,188
6,North RO1,151
7,North RO2,137
8,North RO3,290
9,South RO1,274


In [256]:
Region_final_merg = Merge(Region_enq,'DELR_RGON_NAME',Region_Booking,'DELR_RGON_NAME','','','outer',True,'single')
#Region_final = Hourly_Trend_final[["Range_name","Enquiry Count","Booking Count"]]
Region_final_merg

,DELR_RGON_NAME,ORDER_ID,EQRY_REF_NO,_merge
0,Central RO1,4260,302,both
1,Central RO2,4952,359,both
2,Central RO3,3845,290,both
3,East RO1,2691,218,both
4,East RO2,1032,69,both
5,East RO3,1978,188,both
6,North RO1,2140,151,both
7,North RO2,2393,137,both
8,North RO3,2653,290,both
9,South RO1,3207,274,both


In [313]:
Region_final = Region_final_merg[["DELR_RGON_NAME","ORDER_ID","EQRY_REF_NO"]]
Region_final.columns = ['Dealer Region Name','Enquiry Count','Booking Count']
Region_enq = Region_final["Enquiry Count"].sum()
Region_book = Region_final["Booking Count"].sum()
Region_final["Enquiry Ratio"] = (Region_final["Enquiry Count"]/Region_enq)
Region_final["Booking Ratio"] = (Region_final["Booking Count"]/Region_book)
Region_final["Enq to Booking Ratio"] = (Region_final["Booking Count"]/Region_final["Enquiry Count"])
Region_final["Avg_Enquiry"]=Region_enq/Region_final["Dealer Region Name"].count()
Region_final["Avg_booking"]=Region_book/Region_final["Dealer Region Name"].count()
Region_final

,Dealer Region Name,Enquiry Count,Booking Count,Enquiry Ratio,Booking Ratio,Enq to Booking Ratio,Avg_Enquiry,Avg_booking
0,Central RO1,4260,302,0.075381,0.068934,0.070892,3324.294118,257.705882
1,Central RO2,4952,359,0.087626,0.081945,0.072496,3324.294118,257.705882
2,Central RO3,3845,290,0.068037,0.066195,0.075423,3324.294118,257.705882
3,East RO1,2691,218,0.047617,0.049760,0.081011,3324.294118,257.705882
4,East RO2,1032,69,0.018261,0.015750,0.066860,3324.294118,257.705882
5,East RO3,1978,188,0.035001,0.042913,0.095046,3324.294118,257.705882
6,North RO1,2140,151,0.037867,0.034467,0.070561,3324.294118,257.705882
7,North RO2,2393,137,0.042344,0.031271,0.057250,3324.294118,257.705882
8,North RO3,2653,290,0.046945,0.066195,0.109310,3324.294118,257.705882
9,South RO1,3207,274,0.056748,0.062543,0.085438,3324.294118,257.705882


### Model Wise Analysis

### Zone wise Analysis

In [258]:
GDMS_booking_region = GDMS_booking_add_region_bl.copy()
GDMS_booking_region[["Region Name","RO"]] = GDMS_booking_region.DELR_RGON_NAME.str.split(" ",expand=True)
GDMS_booking_region["Region Name"].count()

4381

In [320]:
def Zone(dataset,element):
    var = dataset[dataset["Region Name"] == element]
    #print(var.shape)
    var = Trend_Analysis_pi(var,['CTB_BOOKING_Y','BKNG_DATE_NEWMonthKey'],'CTB_BOOKING_Y','BKNG_DATE_NEWMonthKey','BKNG_DATE_NEWMonthKey','Monthly')
    var.columns = ['BKNG_DATE_NEWMonthKey', 'Offline Booking', 'Online Booking']
    var["Total Booking"] = var["Offline Booking"]+var["Online Booking"]
    var_off_book = var["Offline Booking"].sum()
    var_on_book = var["Online Booking"].sum()
    var_tot_book = var_off_book+var_on_book
    var_off_book_per = var_off_book/var_tot_book
    var_on_book_per = var_on_book/var_tot_book
    print(element,":- ",var_off_book,var_on_book,var_tot_book,var_off_book_per,var_on_book_per)

    var = var[['BKNG_DATE_NEWMonthKey','Offline Booking', 'Online Booking',"Total Booking"]]
    
    return var

In [321]:
GDMS_booking_north = Zone(GDMS_booking_region,"North")
GDMS_booking_east = Zone(GDMS_booking_region,"East")
GDMS_booking_west = Zone(GDMS_booking_region,"West")
GDMS_booking_south = Zone(GDMS_booking_region,"South")
GDMS_booking_central = Zone(GDMS_booking_region,"Central")

North :-  337 241 578 0.583044982698962 0.41695501730103807
East :-  301 174 475 0.6336842105263157 0.3663157894736842
West :-  761 375 1136 0.6698943661971831 0.3301056338028169
South :-  706 535 1241 0.5688960515713135 0.43110394842868655
Central :-  564 387 951 0.5930599369085173 0.4069400630914827


### Model, Variant, Body and Fuel

In [277]:
def Trend_Analysis1(dataset,Variables, GroupbyVariable):
    #Var_name = TrendType+"_"+dataset
    #count_col_name = EBR_Type+"_"+"Count"
    #print(count_col_name)
    Trend = dataset[Variables].groupby(GroupbyVariable,as_index=False).sum()
    return Trend

In [278]:
def Trend_Analysis2(dataset,Variables, GroupbyVariable):
    #Var_name = TrendType+"_"+dataset
    #count_col_name = EBR_Type+"_"+"Count"
    #print(count_col_name)
    Trend = dataset[Variables].groupby(GroupbyVariable,as_index=False).count()
    return Trend

In [279]:
car_dict = {
    "SUV" : ["New Creta", "Creta", "Venue","Tucson"],
    "HatchBack" : ["Grand i10 NIOS","ELITE i20","All New i20","New i20","Grand i10","Santro","i20 Active"],
    "Sedan" : ["AURA","Elantra","Kona EV","New Verna","Xcent","Next Gen Verna"]
}

body_dict = {}
for k, v in car_dict.items():
    for i in v:
        body_dict[i] = k
body_dict  

{'New Creta': 'SUV',
 'Creta': 'SUV',
 'Venue': 'SUV',
 'Tucson': 'SUV',
 'Grand i10 NIOS': 'HatchBack',
 'ELITE i20': 'HatchBack',
 'All New i20': 'HatchBack',
 'New i20': 'HatchBack',
 'Grand i10': 'HatchBack',
 'Santro': 'HatchBack',
 'i20 Active': 'HatchBack',
 'AURA': 'Sedan',
 'Elantra': 'Sedan',
 'Kona EV': 'Sedan',
 'New Verna': 'Sedan',
 'Xcent': 'Sedan',
 'Next Gen Verna': 'Sedan'}

In [280]:
low = ['AURA_AURA1.2MTCNGS', 'AURA_AURA1.2MTCRDiS', 'AURA_AURA1.2MTKappaE', 'AURA_AURA1.2MTKappaS', 'Creta_CRETA1.4CRDiE+', 'Creta_CRETA1.4CRDiS', 'NewCreta_Creta1.5CRDiMTE', 'NewCreta_Creta1.5CRDiMTS', 'NewCreta_Creta1.5MPiMTE', 'NewCreta_Creta1.5MPiMTS', 'Elantra_ElantraVTVTS', 'Grandi10_Grandi10Magna1.2', 'Grandi10_Grandi10Magna1.2CNG', 'Grandi10NIOS_Grandi10NIOS1.2AMTKappaMagna', 'Grandi10NIOS_Grandi10NIOS1.2MTCNGMagna', 'Grandi10NIOS_Grandi10NIOS1.2MTCorporate', 'Grandi10NIOS_Grandi10NIOS1.2MTCRDiMagna', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaEra', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaMagna', 'i20Active_i20ActiveVTVTS', 'ELITEi20_i20EraCRDi', 'ELITEi20_i20EraVTVT', 'ELITEi20_i20Magna+CRDi', 'ELITEi20_i20Magna+VTVT', 'AllNewi20_MagnaMT', 'Santro_Santro1.1AMTMagna', 'Santro_Santro1.1MTCNGMagna', 'Santro_Santro1.1MTEraExecutive', 'Santro_Santro1.1MTMagna', 'Tucson_TUCSONCRDiAUTOGL(O)', 'Tucson_TUCSONCRDiL', 'Tucson_TUCSONVTVTAUTOGL(O)', 'Tucson_TUCSONVTVTAUTOGL(O)', 'Tucson_TUCSONVTVTL', 'Venue_VENUE1.0TurboGDIDCTS', 'Venue_VENUE1.0TurboGDIMTS', 'Venue_VENUE1.2KappaMTE', 'Venue_VENUE1.2KappaMTS', 'Venue_VENUE1.4CRDiMTE', 'Venue_VENUE1.4CRDiMTS', 'Venue_VENUE1.5CRDiMTE', 'Venue_VENUE1.5CRDiMTS', 'NewVerna_VERNA1.5CRDiMTS+', 'NewVerna_VERNA1.5MPIMTS', 'Xcent_XcentAUTOS']

mid = ['AllNewi20_AstaIVT', 'AllNewi20_AstaMT', 'AllNewi20_AstaTurboDCT', 'AllNewi20_Asta IVT(DualTone)', 'AllNewi20_Asta(O)MT', 'AllNewi20_Asta(O)MT(DualTone)', 'AURA_AURA1.2AMTCRDiS', 'AURA_AURA1.2AMTKappaS', 'AURA_AURA1.2MTCRDiSX(O)', 'AURA_AURA1.2MTKappaSX', 'AURA_AURA1.2MTkappaSX(O)', 'Creta_CRETA1.4CRDiEX', 'NewCreta_Creta1.5CRDiATSX', 'NewCreta_Creta1.5CRDiATSX(O)', 'NewCreta_Creta1.5CRDiMTEX', 'NewCreta_Creta1.5CRDiMTSX', 'NewCreta_Creta1.5CRDiMTSX(O)', 'NewCreta_Creta1.5MPiIVTSX', 'NewCreta_Creta1.5MPiMTEX', 'Creta_CRETA1.6CRDiAUTOSX', 'Creta_CRETA1.6CRDiE+', 'Creta_CRETA1.6CRDiEX', 'Creta_CRETA1.6CRDiSX', 'Creta_CRETA1.6CRDiSX(O)Executive', 'Creta_CRETA1.6VTVTE+', 'Creta_CRETA1.6VTVTSX', 'Creta_CRETA1.6VTVTSX(O)Executive', 'Elantra_ElantraVTVTSX', 'Grandi10NIOS_Grandi10NIOS1.0TurboMTSportz', 'Grandi10NIOS_Grandi10NIOS1.2AMTKappaSportz', 'Grandi10NIOS_Grandi10NIOS1.2AMTCorporate', 'Grandi10NIOS_Grandi10NIOS1.2AMTCRDiSportz', 'Grandi10NIOS_Grandi10NIOS1.2MTCNGSportz', 'Grandi10NIOS_Grandi10NIOS1.2MTCRDiCorporate', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaSportz', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaSportzDT', 'Grandi10_Grandi10PrimeT1.2', 'Grandi10_Grandi10PrimeTCNG', 'Grandi10_Grandi10PrimeTCRDi', 'Grandi10_Grandi10Sportz1.2', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaSportzDT', 'i20Active_i20ActiveCRDiSX', 'ELITEi20_i20Sportz+CRDi', 'ELITEi20_i20Sportz+CVT', 'ELITEi20_i20Sportz+DualtoneCRDi', 'ELITEi20_i20Sportz+DualtoneVTVT', 'ELITEi20_i20Sportz+VTVT', 'Santro_Santro1.1AMTSportz', 'Santro_Santro1.1AMTSportzSE', 'Santro_Santro1.1MTCNGSportz', 'Santro_Santro1.1MTSportz', 'Santro_Santro1.1MTSportzSE', 'AllNewi20_SportzIVT', 'AllNewi20_SportzIVT(DualTone)', 'AllNewi20_SportzMT', 'AllNewi20_SportzMT (DualTone)', 'AllNewi20_SportzMT(DualTone)', 'AllNewi20_SportzTurboiMT', 'AllNewi20_SportzTurboiMT(DualTone)', 'Tucson_TUCSONCRDiAUTOGL', 'Tucson_TUCSONCRDiAUTOGLS', 'Tucson_TUCSONVTVTAUTOGL', 'Tucson_TUCSONVTVTAUTOGLS', 'Venue_VENUE1.0TurboGDIDCTSX+', 'Venue_VENUE1.0TurboGDIiMTSX(O)', 'Venue_VENUE1.0TurboGDIMTSX', 'Venue_VENUE1.0TurboGDIMTSXDualTone', 'Venue_VENUE1.0TurboGDIMTSX(O)', 'Venue_VENUE1.0TurboGDIMTSX(O)DT', 'Venue_VENUE1.0TurboGDIMTSX(O)DualTone', 'Venue_VENUE1.2KappaMTS+', 'Venue_VENUE1.4CRDiMTSX', 'Venue_VENUE1.4CRDiMTSXDualTone', 'Venue_VENUE1.4CRDiMTSX(O)', 'Venue_VENUE1.4CRDiMTSX(O)DT', 'Venue_VENUE1.5CRDiMTSX', 'Venue_VENUE1.5CRDiMTSXDualTone', 'Venue_VENUE1.5CRDiMTSX(O)', 'Venue_VENUE1.5CRDiMTSX(O)DualTone', 'NewVerna_VERNA1.5CRDiATSX', 'NewVerna_VERNA1.5CRDiATSX(O)', 'NewVerna_VERNA1.5CRDiMTSX', 'NewVerna_VERNA1.5CRDiMTSX(O)', 'NewVerna_VERNA1.5MPIIVTSX', 'NewVerna_VERNA1.5MPIIVTSX(O)', 'NewVerna_VERNA1.5MPIMTSX', 'NewVerna_VERNA1.5MPIMTSX(O)', 'NewVerna_Verna1.6CRDiSX', 'NewVerna_Verna1.6CRDiSX(O)', 'NewVerna_Verna1.6VTVTSX', 'NewVerna_Verna1.6VTVTSX(O)', 'Xcent_XcentVTVTPrimeT+CNG']

high = ['AllNewi20_AstaMT(DualTone)', 'AllNewi20_AstaTurboDCT(DualTone)', 'AllNewi20_AstaTurboiMT', 'AllNewi20_AstaTurboiMT(DualTone)', 'AllNewi20_Asta(O)TurboDCT', 'AllNewi20_Asta(O)TurboDCT(DualTone)', 'AURA_AURA1.0TurboGDIMT(SX+)', 'AURA_AURA1.2AMTCRDiSX+', 'AURA_AURA1.2AMTKappaSX+', 'NewCreta_Creta1.4TurboGDiDCTSX', 'NewCreta_Creta1.4TurboGDiDCTSX(O)', 'NewCreta_Creta1.5MPiIVTSX(O)', 'NewCreta_Creta1.5MPiMTSX', 'Creta_CRETA1.6CRDiSXDualTone', 'Creta_CRETA1.6CRDiSXSportsEdition', 'Creta_CRETA1.6CRDiSX(O)', 'Creta_CRETA1.6VTVTAUTOSX', 'Creta_CRETA1.6VTVTSXDualTone', 'Creta_CRETA1.6VTVTSXSportsEdition', 'Creta_CRETA1.6VTVTSX(O)', 'Elantra_ElantraCRDIAutoSX(O)', 'Elantra_ElantraCRDISX', 'Elantra_ElantraVTVTAutoSX', 'Elantra_ElantraVTVTAutoSX(O)', 'Grandi10NIOS_Grandi10NIOS1.2AMTKappaAsta', 'Grandi10NIOS_Grandi10NIOS1.2MTCRDiAsta', 'Grandi10NIOS_Grandi10NIOS1.2MTCRDiSportz', 'Grandi10NIOS_Grandi10NIOS1.2MTKappaAsta', 'ELITEi20_i20Asta(O)CRDi', 'ELITEi20_i20Asta(O)VTVT', 'ELITEi20_i20Asta(O)CVT', 'KonaEV_KonaElectricPremium', 'KonaEV_KonaElectricPremiumDualTone', 'Santro_Santro1.1AMTAsta', 'Santro_Santro1.1MTAsta', 'Tucson_TUCSONCRDi4WDAUTOGLS', 'Venue_VENUE1.0TurboGDIDCTSX+DT', 'Venue_VENUE1.0TurboGDIiMTSXDT', 'Venue_VENUE1.0TurboGDIiMTSX(O)DT', 'NewVerna_VERNA1.0TurboGDIDCTSX(O)', 'NewVerna_Verna1.6CRDiAutoSX(O)', 'NewVerna_Verna1.6CRDiAutoSX+', 'NewVerna_Verna1.6VTVTAutoSX(O)', 'NewVerna_Verna1.6VTVTAutoSX+']

In [281]:
var_type = {"Low": low, "Mid" : mid , "High": high}
#var_dict
var_dict = {}
for k, v in var_type.items():
    for i in v:
        var_dict[i] = k
var_dict

{'AURA_AURA1.2MTCNGS': 'Low',
 'AURA_AURA1.2MTCRDiS': 'Low',
 'AURA_AURA1.2MTKappaE': 'Low',
 'AURA_AURA1.2MTKappaS': 'Low',
 'Creta_CRETA1.4CRDiE+': 'Low',
 'Creta_CRETA1.4CRDiS': 'Low',
 'NewCreta_Creta1.5CRDiMTE': 'Low',
 'NewCreta_Creta1.5CRDiMTS': 'Low',
 'NewCreta_Creta1.5MPiMTE': 'Low',
 'NewCreta_Creta1.5MPiMTS': 'Low',
 'Elantra_ElantraVTVTS': 'Low',
 'Grandi10_Grandi10Magna1.2': 'Low',
 'Grandi10_Grandi10Magna1.2CNG': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2AMTKappaMagna': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2MTCNGMagna': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2MTCorporate': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2MTCRDiMagna': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2MTKappaEra': 'Low',
 'Grandi10NIOS_Grandi10NIOS1.2MTKappaMagna': 'Low',
 'i20Active_i20ActiveVTVTS': 'Low',
 'ELITEi20_i20EraCRDi': 'Low',
 'ELITEi20_i20EraVTVT': 'Low',
 'ELITEi20_i20Magna+CRDi': 'Low',
 'ELITEi20_i20Magna+VTVT': 'Low',
 'AllNewi20_MagnaMT': 'Low',
 'Santro_Santro1.1AMTMagna': 'Low',
 'Santro_Santro1.1MTCNG

In [282]:
BEV_Type_enq =CTB_data.copy()
BEV_Type_enq["model_var_raw"] = BEV_Type_enq["MODEL"] + "_"+BEV_Type_enq["VARIANT"]
BEV_Type_enq = BEV_Type_enq[['ORDER_ID','model_var_raw']].groupby('model_var_raw',as_index=False).count()
print("total enquiry : ",BEV_Type_enq.ORDER_ID.sum())
BEV_Type_enq["model_var"] = BEV_Type_enq.model_var_raw.str.replace(' ', '')
BEV_Type_enq.sort_values(by=['model_var'],inplace=True)
BEV_Type_enq.columns = ["model_var_raw","Enquiry Count","model_var"]
BEV_Type_enq

total enquiry :  56513


,model_var_raw,Enquiry Count,model_var
0,AURA_AURA 1.0 Turbo GDI MT (SX+),53,AURA_AURA1.0TurboGDIMT(SX+)
1,AURA_AURA 1.2AMT CRDi S,33,AURA_AURA1.2AMTCRDiS
2,AURA_AURA 1.2AMT CRDi SX+,41,AURA_AURA1.2AMTCRDiSX+
3,AURA_AURA 1.2AMT Kappa S,278,AURA_AURA1.2AMTKappaS
4,AURA_AURA 1.2AMT Kappa SX+,244,AURA_AURA1.2AMTKappaSX+
...,...,...,...
177,Venue_VENUE 1.5 CRDi MT SX(O),189,Venue_VENUE1.5CRDiMTSX(O)
178,Venue_VENUE 1.5 CRDi MT SX(O) Dual Tone,68,Venue_VENUE1.5CRDiMTSX(O)DualTone
176,Venue_VENUE 1.5 CRDi MT SX Dual Tone,75,Venue_VENUE1.5CRDiMTSXDualTone
179,Xcent_Xcent AUTO S,1,Xcent_XcentAUTOS


In [316]:
BEV_Type_enq[['MODEL','VARIANT']] = BEV_Type_enq["model_var_raw"].str.split("_",expand=True)
BEV_Type_enq["Variant Type"] = BEV_Type_enq["model_var"].map(var_dict)
BEV_Type_enq["Body Type"] = BEV_Type_enq["MODEL"].map(body_dict) 

BEV_Type_enq["Fuel Type"] = 'Petrol'
BEV_Type_enq["Fuel Type"] = np.where(BEV_Type_enq["model_var"].str.contains(pat = 'CRDi'),'Diesel',BEV_Type_enq["Fuel Type"])
BEV_Type_enq["Fuel Type"] = np.where(BEV_Type_enq["model_var"].str.contains(pat = 'CNG'),'CNG',BEV_Type_enq["Fuel Type"])
BEV_Type_enq["Fuel Type"] = np.where(BEV_Type_enq["model_var"].str.contains(pat = 'Electric'),'Electric',BEV_Type_enq["Fuel Type"])
BEV_Type_enq


BEV_Type_enq["MODEL"]=np.where((BEV_Type_enq["MODEL"]=="New i20"),"Elite i20",BEV_Type_enq["MODEL"])
BEV_Type_enq["MODEL"]=np.where((BEV_Type_enq["MODEL"]=="Next Gen Verna"),"New Verna",BEV_Type_enq["MODEL"])



In [1]:
BEV_Type_enq

NameError: name 'BEV_Type_enq' is not defined

In [317]:
variant_enq_final1 = Trend_Analysis1(BEV_Type_enq,['Variant Type','Enquiry Count'],'Variant Type')
body_enq_final1 = Trend_Analysis1(BEV_Type_enq,['Body Type','Enquiry Count'],'Body Type')
fuel_enq_final1 = Trend_Analysis1(BEV_Type_enq,['Fuel Type','Enquiry Count'],'Fuel Type')

model_enq_final1 = Trend_Analysis1(BEV_Type_enq,['MODEL','Enquiry Count'],'MODEL')
model_enq_final1

,MODEL,Enquiry Count
0,AURA,2948
1,All New i20,5340
2,CRETA,175
3,Elantra,183
4,Elite i20,4193
5,Grand i10,1990
6,Grand i10 NIOS,6557
7,I20 ACTIVE,3
8,Kona EV,211
9,New Creta,20196


In [318]:

model_enq_final1["MODEL_lower"] = model_enq_final1["MODEL"].str.lower()
model_enq_final1

,MODEL,Enquiry Count,MODEL_lower
0,AURA,2948,aura
1,All New i20,5340,all new i20
2,CRETA,175,creta
3,Elantra,183,elantra
4,Elite i20,4193,elite i20
5,Grand i10,1990,grand i10
6,Grand i10 NIOS,6557,grand i10 nios
7,I20 ACTIVE,3,i20 active
8,Kona EV,211,kona ev
9,New Creta,20196,new creta


In [286]:
print (fuel_enq_final1)
print (body_enq_final1)
print (variant_enq_final1)

  Fuel Type  Enquiry Count
0       CNG           2010
1    Diesel          12313
2  Electric            211
3    Petrol          41979
   Body Type  Enquiry Count
0  HatchBack          21361
1        SUV          28486
2      Sedan           6488
  Variant Type  Enquiry Count
0         High           7955
1          Low          20110
2          Mid          20933


In [287]:
GDMS_booking.shape

(4381, 27)

In [288]:
GDMS_booking.columns

Index(['CSPM_DLR_NAME', 'EQRY_DLR_NO', 'EQRY_ENQRY_NO', 'EQRY_DATE',
       'EQRY_STAT', 'EQRY_STATUS_NM', 'EQCR_MODEL_CODE', 'EQCR_MODEL_NAME',
       'EQRY_NEWCAR_YN', 'EQRY_REF_NO', 'CTB_BOOKING_Y', 'RETAIL_DATE',
       'INVC_CNFM_YN', 'BKNG_NO', 'BKNG_DATE', 'BKNG_DATE_NEW',
       'BKNG_DATE_NEWYear', 'BKNG_DATE_NEWMonth', 'BKNG_DATE_NEWWeek',
       'BKNG_DATE_NEWDay', 'BKNG_DATE_NEWhour', 'BKNG_DATE_NEWDayofweek',
       'BKNG_DATE_NEWDayofyear', 'BKNG_DATE_NEWweekofyear',
       'BKNG_DATE_NEWMonthName', 'BKNG_DATE_NEWMonthKey',
       'BKNG_DATE_NEWWeekKey'],
      dtype='object')

In [289]:
BEVM_Type_book = pd.merge(CTB_data_raw, GDMS_booking, left_on=['ORDER_ID'],right_on = ['EQRY_REF_NO'], how = 'outer',indicator=True)
print("BEVM_Type_book" , BEVM_Type_book.shape)
BEVM_Type_book['_merge'].value_counts()

BEVM_Type_book (64454, 72)


left_only     60049
both           4396
right_only        9
Name: _merge, dtype: int64

In [291]:
BEVM_Type_book_match = BEVM_Type_book[BEVM_Type_book['_merge']!="left_only"][["ORDER_ID","MODEL","VARIANT","EQRY_ENQRY_NO","EQCR_MODEL_NAME","BKNG_DATE","CTB_BOOKING_Y","RETAIL_DATE","EQRY_STAT","_merge"]]
BEVM_Type_book_match

,ORDER_ID,MODEL,VARIANT,EQRY_ENQRY_NO,EQCR_MODEL_NAME,BKNG_DATE,CTB_BOOKING_Y,RETAIL_DATE,EQRY_STAT,_merge
29,ORD202012587646762,Venue,VENUE 1.4 CRDi MT E,E202004115,Venue,20200205.0,N,20200207.0,45.0,both
111,ORD202021623878191,New i20,i20 Asta(O) CVT,E202007954,Next Gen Verna,20200219.0,N,NaN,60.0,both
228,ORD2020122016767358,New Creta,Creta 1.5 CRDi MT E,E202049255,New Creta,20201224.0,N,NaN,20.0,both
308,ORD2020062816899997,New Creta,Creta 1.5 MPi IVT SX(O),E202021186,New Creta,20200705.0,N,20200804.0,45.0,both
317,ORD2021013115955093,New Creta,Creta 1.5 MPi MT E,E202105132,New Creta,20210131.0,Y,NaN,20.0,both
...,...,...,...,...,...,...,...,...,...,...
64449,NaN,NaN,NaN,E202025890,New Creta,20200806.0,N,20200822.0,45.0,right_only
64450,NaN,NaN,NaN,E202021760,Venue,20200814.0,N,20200814.0,45.0,right_only
64451,NaN,NaN,NaN,E202025900,New Creta,20201002.0,N,NaN,60.0,right_only
64452,NaN,NaN,NaN,E202011382,New Creta,20200921.0,Y,NaN,91.0,right_only


In [292]:
BEVM_Type_book_match1 = BEVM_Type_book_match.drop_duplicates(subset=['EQRY_ENQRY_NO',"ORDER_ID"], keep='last')
BEVM_Type_book_match1.shape

(4381, 10)

In [293]:
BEVM_Type_book_match1.head()

,ORDER_ID,MODEL,VARIANT,EQRY_ENQRY_NO,EQCR_MODEL_NAME,BKNG_DATE,CTB_BOOKING_Y,RETAIL_DATE,EQRY_STAT,_merge
29,ORD202012587646762,Venue,VENUE 1.4 CRDi MT E,E202004115,Venue,20200205.0,N,20200207.0,45.0,both
111,ORD202021623878191,New i20,i20 Asta(O) CVT,E202007954,Next Gen Verna,20200219.0,N,NaN,60.0,both
228,ORD2020122016767358,New Creta,Creta 1.5 CRDi MT E,E202049255,New Creta,20201224.0,N,NaN,20.0,both
308,ORD2020062816899997,New Creta,Creta 1.5 MPi IVT SX(O),E202021186,New Creta,20200705.0,N,20200804.0,45.0,both
317,ORD2021013115955093,New Creta,Creta 1.5 MPi MT E,E202105132,New Creta,20210131.0,Y,NaN,20.0,both


In [294]:
#BEVM_Type_book_match1[['MODEL','VARIANT']] = BEVM_Type_book_match1["model_var"].str.split("_",expand=True)
BEVM_Type_book_match1["model_var"] = BEVM_Type_book_match1["MODEL"] + "_"+BEVM_Type_book_match1["VARIANT"]
BEVM_Type_book_match1.model_var = BEVM_Type_book_match1.model_var.str.replace(' ', '')
BEVM_Type_book_match1["Variant Type"] = BEVM_Type_book_match1["model_var"].map(var_dict)
BEVM_Type_book_match1["Body Type"] = BEVM_Type_book_match1["EQCR_MODEL_NAME"].map(body_dict) 

BEVM_Type_book_match1["Fuel Type"] = 'Petrol'
BEVM_Type_book_match1["Fuel Type"] = np.where(BEVM_Type_book_match1["model_var"].str.contains(pat = 'CRDi'),'Diesel',BEVM_Type_book_match1["Fuel Type"])
BEVM_Type_book_match1["Fuel Type"] = np.where(BEVM_Type_book_match1["model_var"].str.contains(pat = 'CNG'),'CNG',BEVM_Type_book_match1["Fuel Type"])
BEVM_Type_book_match1["Fuel Type"] = np.where(BEVM_Type_book_match1["model_var"].str.contains(pat = 'Electric'),'Electric',BEVM_Type_book_match1["Fuel Type"])
BEVM_Type_book_match1["Fuel Type"] = np.where(BEVM_Type_book_match1["model_var"].str.contains(pat = 'NULL'),'Petrol',BEVM_Type_book_match1["Fuel Type"])

BEVM_Type_book_match1["EQCR_MODEL_NAME"]=np.where((BEVM_Type_book_match1["EQCR_MODEL_NAME"]=="New i20"),"All New i20",BEVM_Type_book_match1["EQCR_MODEL_NAME"])
BEVM_Type_book_match1["EQCR_MODEL_NAME"]=np.where((BEVM_Type_book_match1["EQCR_MODEL_NAME"]=="Next Gen Verna"),"New Verna",BEVM_Type_book_match1["EQCR_MODEL_NAME"])


BEVM_Type_book_match1

,ORDER_ID,MODEL,VARIANT,EQRY_ENQRY_NO,EQCR_MODEL_NAME,BKNG_DATE,CTB_BOOKING_Y,RETAIL_DATE,EQRY_STAT,_merge,model_var,Variant Type,Body Type,Fuel Type
29,ORD202012587646762,Venue,VENUE 1.4 CRDi MT E,E202004115,Venue,20200205.0,N,20200207.0,45.0,both,Venue_VENUE1.4CRDiMTE,Low,SUV,Diesel
111,ORD202021623878191,New i20,i20 Asta(O) CVT,E202007954,New Verna,20200219.0,N,NaN,60.0,both,Newi20_i20Asta(O)CVT,NaN,Sedan,Petrol
228,ORD2020122016767358,New Creta,Creta 1.5 CRDi MT E,E202049255,New Creta,20201224.0,N,NaN,20.0,both,NewCreta_Creta1.5CRDiMTE,Low,SUV,Diesel
308,ORD2020062816899997,New Creta,Creta 1.5 MPi IVT SX(O),E202021186,New Creta,20200705.0,N,20200804.0,45.0,both,NewCreta_Creta1.5MPiIVTSX(O),High,SUV,Petrol
317,ORD2021013115955093,New Creta,Creta 1.5 MPi MT E,E202105132,New Creta,20210131.0,Y,NaN,20.0,both,NewCreta_Creta1.5MPiMTE,Low,SUV,Petrol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64449,NaN,NaN,NaN,E202025890,New Creta,20200806.0,N,20200822.0,45.0,right_only,NaN,NaN,SUV,Petrol
64450,NaN,NaN,NaN,E202021760,Venue,20200814.0,N,20200814.0,45.0,right_only,NaN,NaN,SUV,Petrol
64451,NaN,NaN,NaN,E202025900,New Creta,20201002.0,N,NaN,60.0,right_only,NaN,NaN,SUV,Petrol
64452,NaN,NaN,NaN,E202011382,New Creta,20200921.0,Y,NaN,91.0,right_only,NaN,NaN,SUV,Petrol


In [295]:
BEVM_Type_book_match2 = BEVM_Type_book_match1[BEVM_Type_book_match1['_merge']=="right_only"]
BEVM_Type_book_match2

,ORDER_ID,MODEL,VARIANT,EQRY_ENQRY_NO,EQCR_MODEL_NAME,BKNG_DATE,CTB_BOOKING_Y,RETAIL_DATE,EQRY_STAT,_merge,model_var,Variant Type,Body Type,Fuel Type
64445,NaN,NaN,NaN,E202020927,New Creta,20200626.0,Y,NaN,91.0,right_only,NaN,NaN,SUV,Petrol
64446,NaN,NaN,NaN,E202015348,New Creta,20200506.0,Y,20200629.0,45.0,right_only,NaN,NaN,SUV,Petrol
64447,NaN,NaN,NaN,E202026223,New Creta,20200805.0,N,NaN,60.0,right_only,NaN,NaN,SUV,Petrol
64448,NaN,NaN,NaN,E202027099,AURA,20200818.0,N,20200911.0,45.0,right_only,NaN,NaN,Sedan,Petrol
64449,NaN,NaN,NaN,E202025890,New Creta,20200806.0,N,20200822.0,45.0,right_only,NaN,NaN,SUV,Petrol
64450,NaN,NaN,NaN,E202021760,Venue,20200814.0,N,20200814.0,45.0,right_only,NaN,NaN,SUV,Petrol
64451,NaN,NaN,NaN,E202025900,New Creta,20201002.0,N,NaN,60.0,right_only,NaN,NaN,SUV,Petrol
64452,NaN,NaN,NaN,E202011382,New Creta,20200921.0,Y,NaN,91.0,right_only,NaN,NaN,SUV,Petrol
64453,NaN,NaN,NaN,E202027539,New Creta,20200827.0,N,20201023.0,45.0,right_only,NaN,NaN,SUV,Petrol


In [296]:
variant_book_final1 = Trend_Analysis2(BEVM_Type_book_match1,['Variant Type','ORDER_ID'],'Variant Type')
body_book_final1 = Trend_Analysis2(BEVM_Type_book_match1,['Body Type','EQRY_ENQRY_NO'],'Body Type')
fuel_book_final1 = Trend_Analysis2(BEVM_Type_book_match1,['Fuel Type','ORDER_ID'],'Fuel Type')
model_book_final1 = Trend_Analysis2(BEVM_Type_book_match1,['EQCR_MODEL_NAME','EQRY_ENQRY_NO'],'EQCR_MODEL_NAME')
model_book_final1

,EQCR_MODEL_NAME,EQRY_ENQRY_NO
0,AURA,208
1,All New i20,1096
2,Creta,3
3,Elantra,2
4,Grand i10,32
5,Grand i10 NIOS,412
6,Kona EV,1
7,New Creta,1876
8,New Verna,138
9,Santro,100


In [297]:
model_book_final1["MODEL_lower"] = model_book_final1["EQCR_MODEL_NAME"].str.lower()
model_book_final1

,EQCR_MODEL_NAME,EQRY_ENQRY_NO,MODEL_lower
0,AURA,208,aura
1,All New i20,1096,all new i20
2,Creta,3,creta
3,Elantra,2,elantra
4,Grand i10,32,grand i10
5,Grand i10 NIOS,412,grand i10 nios
6,Kona EV,1,kona ev
7,New Creta,1876,new creta
8,New Verna,138,new verna
9,Santro,100,santro


In [298]:
print(fuel_book_final1)
print(body_book_final1)
print(variant_book_final1)

  Fuel Type  ORDER_ID
0       CNG       136
1    Diesel      1051
2  Electric         2
3    Petrol      3183
   Body Type  EQRY_ENQRY_NO
0  HatchBack           1640
1        SUV           2391
2      Sedan            350
  Variant Type  ORDER_ID
0         High       791
1          Low      1328
2          Mid      1896


In [300]:
variant_final = Merge(variant_enq_final1,'Variant Type',variant_book_final1,'Variant Type','','','outer',False,'single')
variant_final.columns =['Variant Type', 'Enquiry Count', 'Booking Count']

total_enq = variant_final["Enquiry Count"].sum()
total_book = variant_final["Booking Count"].sum()

variant_final["Enquiry Ratio"] = (variant_final["Enquiry Count"]/total_enq)
variant_final["Booking Ratio"] = (variant_final["Booking Count"]/total_book)
variant_final["Enq to Booking Ratio"] = (variant_final["Booking Count"]/total_enq)
variant_final

,Variant Type,Enquiry Count,Booking Count,Enquiry Ratio,Booking Ratio,Enq to Booking Ratio
0,High,7955,791,0.162354,0.197011,0.016144
1,Low,20110,1328,0.410425,0.330760,0.027103
2,Mid,20933,1896,0.427222,0.472229,0.038695


In [301]:
body_final = Merge(body_enq_final1,'Body Type',body_book_final1,'Body Type','','','outer',False,'single')
body_final.columns =['Body Type', 'Enquiry Count', 'Booking Count']

total_enq = body_final["Enquiry Count"].sum()
total_book = body_final["Booking Count"].sum()

body_final["Enquiry Ratio"] = (body_final["Enquiry Count"]/total_enq)
body_final["Booking Ratio"] = (body_final["Booking Count"]/total_book)
body_final["Enq to Booking Ratio"] = (body_final["Booking Count"]/total_enq)
body_final

,Body Type,Enquiry Count,Booking Count,Enquiry Ratio,Booking Ratio,Enq to Booking Ratio
0,HatchBack,21361,1640,0.379178,0.374344,0.029112
1,SUV,28486,2391,0.505654,0.545766,0.042443
2,Sedan,6488,350,0.115168,0.079890,0.006213


In [302]:
fuel_final = Merge(fuel_enq_final1,'Fuel Type',fuel_book_final1,'Fuel Type','','','outer',False,'single')
fuel_final.columns =['Fuel Type', 'Enquiry Count', 'Booking Count']

total_enq = fuel_final["Enquiry Count"].sum()
total_book = fuel_final["Booking Count"].sum()

fuel_final["Enquiry Ratio"] = (fuel_final["Enquiry Count"]/total_enq)
fuel_final["Booking Ratio"] = (fuel_final["Booking Count"]/total_book)
fuel_final["Enq to Booking Ratio"] = (fuel_final["Booking Count"]/total_enq)
fuel_final

,Fuel Type,Enquiry Count,Booking Count,Enquiry Ratio,Booking Ratio,Enq to Booking Ratio
0,CNG,2010,136,0.035567,0.031107,0.002407
1,Diesel,12313,1051,0.217879,0.240393,0.018597
2,Electric,211,2,0.003734,0.000457,0.000035
3,Petrol,41979,3183,0.742820,0.728042,0.056323


In [319]:
model_final_mer = Merge(model_enq_final1,'MODEL_lower',model_book_final1,'MODEL_lower','','','outer',True,'single')
model_final_mer.columns =['MODEL', 'Enquiry Count','MODEL_lower', 'EQCR_MODEL_NAME', 'Booking Count', '_merge']
model_final = model_final_mer[['MODEL','EQCR_MODEL_NAME', 'Enquiry Count', 'Booking Count', '_merge']]
model_final["Enq to Booking Ratio"] = (model_final["Booking Count"]/model_final["Enquiry Count"])
#model_final.fillna("0", inplace = True)
model_final

,MODEL,EQCR_MODEL_NAME,Enquiry Count,Booking Count,_merge,Enq to Booking Ratio
0,AURA,AURA,2948,208.0,both,0.070556
1,All New i20,All New i20,5340,1096.0,both,0.205243
2,CRETA,Creta,175,3.0,both,0.017143
3,Elantra,Elantra,183,2.0,both,0.010929
4,Elite i20,NaN,4193,NaN,left_only,NaN
5,Grand i10,Grand i10,1990,32.0,both,0.016080
6,Grand i10 NIOS,Grand i10 NIOS,6557,412.0,both,0.062834
7,I20 ACTIVE,NaN,3,NaN,left_only,NaN
8,Kona EV,Kona EV,211,1.0,both,0.004739
9,New Creta,New Creta,20196,1876.0,both,0.092890


### Save in excel sheet

In [312]:
writer = pd.ExcelWriter('CTB_auto_monthly_report_08Feb.xlsx', engine='xlsxwriter')

Monthly_Trend_final.to_excel(writer, sheet_name='Monthly_Trend_final')
Weekly_Trend_final.to_excel(writer, sheet_name='Weekly_Trend_final')
Hourly_Trend_final.to_excel(writer, sheet_name='Hourly_Trend_final')
Region_final.to_excel(writer, sheet_name='Region_final')
model_final.to_excel(writer, sheet_name='model_final')


variant_final.to_excel(writer, sheet_name='variant_final')
body_final.to_excel(writer, sheet_name='body_final')
fuel_final.to_excel(writer, sheet_name='fuel_final')
model_final.to_excel(writer, sheet_name='model_final')


GDMS_booking_north.to_excel(writer, sheet_name='GDMS_booking_north')
GDMS_booking_east.to_excel(writer, sheet_name='GDMS_booking_east')
GDMS_booking_west.to_excel(writer, sheet_name='GDMS_booking_west')
GDMS_booking_south.to_excel(writer, sheet_name='GDMS_booking_south')
GDMS_booking_central.to_excel(writer, sheet_name='GDMS_booking_central')



writer.save()

In [322]:
writer = pd.ExcelWriter('CTB_auto_monthly_report_08Feb_Zone.xlsx', engine='xlsxwriter')

GDMS_booking_north.to_excel(writer, sheet_name='GDMS_booking_north')
GDMS_booking_east.to_excel(writer, sheet_name='GDMS_booking_east')
GDMS_booking_west.to_excel(writer, sheet_name='GDMS_booking_west')
GDMS_booking_south.to_excel(writer, sheet_name='GDMS_booking_south')
GDMS_booking_central.to_excel(writer, sheet_name='GDMS_booking_central')



writer.save()

In [326]:
GCRM3 = pd.read_csv("CTB_Jan_cust.csv",sep='\t', lineterminator='\r')
print(GCRM3.shape)
print(GCRM3.columns)
GCRM3.head(2)

(8742, 6)


,LAST_NAME,CELL_PH_NUM,EMAIL_ADDR,SEX_MF,BIRTH_DT,OCCUPATION
0,\nDUMMY,1.234568e+09,dummy@gmail.com,M,NaN,Other
1,\nNIMESHBHAI BABULAL SHAH SOLD CAR,1.234568e+09,nil456@gmail.com,M,NaN,NaN


In [333]:
GCRM3.to_excel("GCRM_Jan_Month_spRemove.xlsx")

Index(['LAST_NAME', 'CELL_PH_NUM', 'EMAIL_ADDR', 'SEX_MF', 'BIRTH_DT',
       'OCCUPATION'],
      dtype='object')

In [332]:
GCRM3['Last_name'] = GCRM3['LAST_NAME'].str[1:]
GCRM3.head()

,LAST_NAME,CELL_PH_NUM,EMAIL_ADDR,SEX_MF,BIRTH_DT,OCCUPATION,Last_name
0,\nDUMMY,1.234568e+09,dummy@gmail.com,M,NaN,Other,DUMMY
1,\nNIMESHBHAI BABULAL SHAH SOLD CAR,1.234568e+09,nil456@gmail.com,M,NaN,NaN,NIMESHBHAI BABULAL SHAH SOLD CAR
2,\nMAHENDRSINGA BHARATSINHA RANA SOLD CAR,1.234568e+09,nil456@gmail.com,M,NaN,NaN,MAHENDRSINGA BHARATSINHA RANA SOLD CAR
3,\nANUSHA DEEPAK TYAGI,1.234568e+09,s@s.com,M,NaN,NaN,ANUSHA DEEPAK TYAGI
4,\nVIKAS KUMAR 0,1.234568e+09,vikas456@gmail.com,M,NaN,NaN,VIKAS KUMAR 0


In [ ]:
GCRM3.

In [336]:
data = pd.read_excel("D:\\Work\\Worksheet in 20210107_CTB BigData Way forward.xltm")

In [339]:
data.to_excel("excel.xlsx")